In [89]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
!pip install yfinance -q

import yfinance as yf

In [5]:
#python должен быть версии 3.6-3.9 для установки numpy==1.19.5. Эта версия нужна для работы ta-lib
!pip uninstall -y numpy
!pip install numpy==1.24.4 -q

import numpy as np

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4


In [6]:
# работа выполнялась на MacOS 10.15
# предварительно запустить brew install ta-lib
!pip install TA-Lib -q

import talib

In [51]:
!pip install plotly -q

import plotly.graph_objects as go

def linear_plot(df, title):
    fig = go.Figure([go.Scatter(x=df['date'], y=df['close'], mode='lines')])
    fig.update_layout(plot_bgcolor='white', 
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.show()

def candlestick_plot(df, title):
    fig = go.Figure([go.Candlestick(x=df['date'], 
                                open=df['open'],
                                high=df['high'],
                                low=df['low'],
                                close=df['close'])])
    fig.update_layout(xaxis_rangeslider_visible=False,
                      plot_bgcolor='white',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.update_yaxes(fixedrange=False)
    fig.show()
    

In [73]:
import pandas as pd

pd.set_option('display.max_rows', 100)

In [78]:
df = yf.download("BTC-USD", start='2020-01-01', interval='1d').drop(columns=['Adj Close'])
df = df.reset_index()
df.columns = df.columns.str.lower()
df

[*********************100%***********************]  1 of 1 completed


,date,close,high,low,open,volume
0,2020-01-01 00:00:00+00:00,7200.174316,7254.330566,7174.944336,7194.892090,18565664997
1,2020-01-02 00:00:00+00:00,6985.470215,7212.155273,6935.270020,7202.551270,20802083465
2,2020-01-03 00:00:00+00:00,7344.884277,7413.715332,6914.996094,6984.428711,28111481032
3,2020-01-04 00:00:00+00:00,7410.656738,7427.385742,7309.514160,7345.375488,18444271275
4,2020-01-05 00:00:00+00:00,7411.317383,7544.497070,7400.535645,7410.451660,19725074095
...,...,...,...,...,...,...
1756,2024-10-22 00:00:00+00:00,67361.406250,67801.578125,66581.367188,67360.703125,31808472566
1757,2024-10-23 00:00:00+00:00,66432.195312,67402.742188,65188.035156,67362.375000,32263980353
1758,2024-10-24 00:00:00+00:00,68161.054688,68798.960938,66454.101562,66653.703125,31414428647
1759,2024-10-25 00:00:00+00:00,66642.414062,68722.156250,65521.792969,68165.296875,41469984306


In [57]:
linear_plot(df, 'BTC/USD')

In [79]:
bt_df_sma = df.copy()
#bt_df_sma = bt_df_sma.reset_index()
bt_df_sma.columns = bt_df_sma.columns.str.capitalize()
bt_df_sma.rename(columns={'Date': 'Datetime'}, inplace=True)
bt_df_sma["Datetime"] = pd.to_datetime(bt_df_sma["Datetime"])
bt_df_sma.set_index('Datetime', inplace=True)
bt_df_sma

,Close,High,Low,Open,Volume
Datetime,,,,,
2020-01-01 00:00:00+00:00,7200.174316,7254.330566,7174.944336,7194.892090,18565664997
2020-01-02 00:00:00+00:00,6985.470215,7212.155273,6935.270020,7202.551270,20802083465
2020-01-03 00:00:00+00:00,7344.884277,7413.715332,6914.996094,6984.428711,28111481032
2020-01-04 00:00:00+00:00,7410.656738,7427.385742,7309.514160,7345.375488,18444271275
2020-01-05 00:00:00+00:00,7411.317383,7544.497070,7400.535645,7410.451660,19725074095
...,...,...,...,...,...
2024-10-22 00:00:00+00:00,67361.406250,67801.578125,66581.367188,67360.703125,31808472566
2024-10-23 00:00:00+00:00,66432.195312,67402.742188,65188.035156,67362.375000,32263980353
2024-10-24 00:00:00+00:00,68161.054688,68798.960938,66454.101562,66653.703125,31414428647


In [91]:
!pip install backtesting -q

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

Попробуем простую стратегию на основе скользящих средних из документации к библиотеке Backtesting (https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html)

In [81]:
# def SMA(values, n):
#     """
#     Return simple moving average of `values`, at
#     each step taking into account `n` previous values.
#     """
#     return pd.Series(values).rolling(n).mean()

In [102]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [103]:
bt_sma = Backtest(bt_df_sma, SmaCross, cash=1_000_000, commission=.002, exclusive_orders=True)
stats = bt_sma.run()
print(stats)

Start                     2020-01-01 00:00...
End                       2024-10-26 00:00...
Duration                   1760 days 00:00:00
Exposure Time [%]                    96.93356
Equity Final [$]                719242.069898
Equity Peak [$]                6130124.633664
Return [%]                         -28.075793
Buy & Hold Return [%]              828.691709
Return (Ann.) [%]                   -6.602619
Volatility (Ann.) [%]               59.873362
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -91.929825
Avg. Drawdown [%]                   -8.820616
Max. Drawdown Duration     1387 days 00:00:00
Avg. Drawdown Duration       76 days 00:00:00
# Trades                                   92
Win Rate [%]                        33.695652
Best Trade [%]                     181.445437
Worst Trade [%]                    -22.985365
Avg. Trade [%]                    

In [94]:
bt_sma.plot()

GridPlot(id='p3764', ...)

По результатам бэктестинга мы получили убыток.
Попробуем сделать оптимизацию и улучшить наши результаты

In [97]:
stats = bt_sma.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
print(stats)

Start                     2020-01-01 00:00...
End                       2024-10-26 00:00...
Duration                   1760 days 00:00:00
Exposure Time [%]                   96.592845
Equity Final [$]               8192999.754398
Equity Peak [$]               16240972.673984
Return [%]                         719.299975
Buy & Hold Return [%]              828.691709
Return (Ann.) [%]                   54.642206
Volatility (Ann.) [%]               96.787818
Sharpe Ratio                         0.564557
Sortino Ratio                        1.467013
Calmar Ratio                         0.979244
Max. Drawdown [%]                  -55.800403
Avg. Drawdown [%]                   -9.934571
Max. Drawdown Duration      681 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                   42
Win Rate [%]                        42.857143
Best Trade [%]                      348.01875
Worst Trade [%]                    -20.442894
Avg. Trade [%]                    

In [98]:
stats._strategy

<Strategy SmaCross(n1=10,n2=45)>

In [99]:
bt_sma.plot()

GridPlot(id='p4890', ...)

Оптимизация параметров окна скользящей средней позволила получить прибыль

Попробуем применить индикаторы технического анализа из лекций и посмотрим на результаты

In [ ]:
class TemaMacdStrategy(Strategy):
    def init(self):
        self.signal = self.I(lambda: self.data.Signal)
        self.previous_signal = 0

    def next(self):
        current_signal = self.signal[-1]

        if current_signal != self.previous_signal:
            if current_signal == 1:
                if self.position.is_short:
                    self.position.close()
                    
                if not self.position.is_long:
                    self.buy()
                    
            elif current_signal == -1:
                if self.position.is_long:
                    self.position.close()
                   
                if not self.position.is_short:
                    self.sell()
                    
            elif current_signal == 0:
                if self.position:
                    self.position.close()
                    

        self.previous_signal = current_signal

In [17]:
# Разделим датафрейм на обучающую и тестовую выборки
split_value = int(len(df) * 0.7)
train_df = df.iloc[:split_value]
test_df = df.iloc[split_value:].reset_index(drop=True)

In [18]:
# Подготовка данных для бэктестинга
bt_df = test_df.copy()
bt_df.columns = bt_df.columns.str.capitalize()
bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
bt_df.set_index('Datetime', inplace=True)

In [20]:
bt_df.dtypes

Close     float64
High      float64
Low       float64
Open      float64
Volume      int64
dtype: object

In [38]:
bt_df

,Close,High,Low,Open,Volume
Datetime,,,,,
2023-12-22 00:00:00+00:00,43997.902344,44367.957031,43441.968750,43868.988281,21028503216
2023-12-23 00:00:00+00:00,43739.542969,44015.699219,43351.355469,44012.199219,13507796558
2023-12-24 00:00:00+00:00,43016.117188,43945.523438,42786.917969,43728.367188,18830554085
2023-12-25 00:00:00+00:00,43613.140625,43765.093750,42765.769531,43010.574219,21115795370
2023-12-26 00:00:00+00:00,42520.402344,43603.175781,41676.488281,43599.847656,30026850982
...,...,...,...,...,...
2024-10-21 00:00:00+00:00,67367.851562,69462.734375,66829.851562,69002.000000,37498611780
2024-10-22 00:00:00+00:00,67361.406250,67801.578125,66581.367188,67360.703125,31808472566
2024-10-23 00:00:00+00:00,66432.195312,67402.742188,65188.035156,67362.375000,32263980353
